In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import pickle

In [2]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

# Audf

In [3]:
with open('../input/entities/entityAu.pkl', 'rb') as f:
    entitiesA = pickle.load(f)

In [4]:
Per=[]
Date=[]
GPE=[]
ORG=[]
from flair.data import Sentence

Audf=pd.DataFrame({"text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})



for i in entitiesA:
    
 dc={}
 P=[]
 D=[]
 G=[]
 O=[]
 for entity in i:
    if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        
        
        
 for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
 Per.append(P)
 Date.append(D)
 ORG.append(O)
 GPE.append(G)
            
Audf["Date-ontonotes"]=Date
Audf["PERSON-ontonotes"]=Per
Audf["GPE-ontonotes"]=GPE
Audf["ORG-ontonotes"]=ORG    

In [5]:
Audf

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,NaN,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]"
1,NaN,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864]
2,NaN,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]"
3,NaN,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]"
4,NaN,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]"
...,...,...,...,...,...
2131,NaN,"[Geo . Little, Geo . B. Ayer]","[Raleigh, N.C., Wake County, N.C, United State...",[],"[May 26, July 14]"
2132,NaN,[Birnie],[],[],[July 30th 1867]
2133,NaN,[William Birnie],"[North Carolina, Robeson County, United States]",[],[30 day of July]
2134,NaN,"[A.L . Price, James Fulton, John D Conoley, Al...","[North Carolina, New Hanover County, New Hanov...",[Fulton Price],"[May 29 1865, 15th Sept 1865, 15th September A..."


In [6]:
G=[]
for i in Audf["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-ZZa-z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)

    
Audf["postDate-ontonotes"]=GF

In [7]:
G=[]
for i in Audf["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

Audf["postDate-ontonotes"]=G

In [8]:
NG=[]
for i in G:
    ng=[]
    for j in i:
        
        date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',j)
        if date !=[]:
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)
                
                
                if dates[1] in ["August","Aug","augt","AUGUST","Augst","Augt","AUG","Augusta","Aug.",'Agusut']:
                    dates[1]= "August"
                elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER",'Sepr','Septh',"Sep.","Sept."]:
                    dates[1]= "Septembre"
                elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                    dates[1]= "October"
                elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER", "The","Dec"]:
                    dates[1]= "November"
                elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber",'Dec.']:
                    dates[1]= "December"
                elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY",'Jan.']:
                    dates[1]= "January"
                elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY","Feb"]:
                    dates[1]= "February"
                elif dates[1] in ["March","Mar"]:
                    dates[1]= "March"
                elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                    dates[1]= "April"
                elif dates[1] in ["May","MAY"]:
                    dates[1]= "May"
                elif dates[1] in ["JUN","Jun","June"]:
                    dates[1]= "June"
                elif dates[1] in ["JULY","July","JUL"]:
                    dates[1]= "July"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
        else :
                date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})',j ) 
                dates=[]
                for k in date:
                    for l in k:
                        dates.append(l)
                
                
                    if dates[1] in ["August","Aug","augt","AUGUST","Augst",'Augt',"AUG","Augusta","Aug.","Agusut"]:
                        dates[1]= "August"
                    elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER","Sepr","Septh", "This", "This ","Sept.","Sep."]:
                        dates[1]= "Septembre"
                    elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                        dates[1]= "October"
                    elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
                        dates[1]= "November"
                    elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber","Dec."]:
                        dates[1]= "December"
                    elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY","Jan."]:
                        dates[1]= "January"
                    elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY","Feb","Feb."]:
                        dates[1]= "February"
                    elif dates[1] in ["March","Mar"]:
                        dates[1]= "March"
                    elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                        dates[1]= "April"
                    elif dates[1] in ["May","MAY"]:
                        dates[1]= "May"
                    elif dates[1] in ["JUN","Jun","June"]:
                        dates[1]= "June"
                    elif dates[1] in ["JULY","July","JUL"]:
                        dates[1]= "July"
                    if dates[0]== "fifteenth" :
                        dates[0]="15"
                    elif (dates[0]=="sixteenth") |(dates[0]=="This 16 "):
                        dates[0]="16"
                    elif dates[0]=="Twenty":
                        dates[0]="20" 
                    elif dates[0]=="Twentyseventh":
                        dates[0]="27" 
                    elif (dates[0]=="Twenty Eighth") |(dates[0]=="TwentyEight") |(dates[0]=="TwentyEight ") |(dates[0]=="Twentyeight"):
                        dates[0]="28"
                    elif (dates[0]=="Ninth")|(dates[0]=="The 9th ") |(dates[0]=="Ninth ")|(dates[0]=="Augusta 9th ") |(dates[0]=="ninth") |(dates[0]=="ninth ")|(dates[0]=="Augusta 9th"):
                        dates[0]="9"
                    
                    elif dates[0]=='Seventh':
                        dates[0]="7" 
                    elif (dates[0]=='The') |(dates[0]=="The 9th") :
                        dates[0]="9"
                    elif dates[0]=="the 9" :
                        dates[0]="9"
                    elif (dates[0]=="Eighth ") |(dates[0]=="Eighth") |(dates[0]=="eighth") |(dates[0]=="eighth "):
                        dates[0]="8"
                    elif dates[0]=="Seventh ":
                        dates[0]="7"
                    elif (dates[0]=="Twenty ninth ")| (dates[0]=="Twenty ninth") | (dates[0]=="twenty ninth ")|(dates[0]=="Twenty Ninth ")|(dates[0]=="Twenty Ninth") :
                        dates[0]="29"
                    elif (dates[0]=="Thirty") |(dates[0]=="Thirty ") | (dates[0]=="thirtieth "):
                        dates[0]="30"
                    elif (dates[0]=="Thirty first ") |(dates[0]=="Thirty first"):
                        dates[0]="31"
                  
                    
                    elif dates[0]=="first ":
                        dates[0]="1"
                    elif (dates[0]=="Second ") |(dates[0]=="Second") |(dates[0]=="second")|(dates[0]=="second "):
                        dates[0]="2"
                    elif dates[0]=="fourth ":
                        dates[0]="4"
                    elif dates[0]=="tenth ":
                        dates[0]="10"
                    elif (dates[0]=="Fifth ")|(dates[0]=="Fifth") |(dates[0]=="fifth ")|(dates[0]=="fifth"):
                        dates[0]="5"
                    elif (dates[0]=="This") | (dates[0]=="This 7th ")|(dates[0]=="seventh ") |(dates[0]=="seventh") |(dates[0]=="Seventh "):
                        dates[0]="7"
                    elif (dates[0]=="This 31st") |(dates[0]=="Thirty-first "):
                        dates[0]="31"
                    elif (dates[0]=="Thirtieth") |(dates[0]=="Thirtieth ") |(dates[0]=="Thirty"):
                        dates[0]="30"
                    elif dates[0]=="twenty fifth ":
                        dates[0]="25"
                    elif dates[0]=="Twenty seventh ":
                        dates[0]="27"
                    elif (dates[0]=="Twenty sixth ") |(dates[0]=="Twentysixth ")|(dates[0]=="Twentysixth") |(dates[0]=="twentysixth "):
                        dates[0]="26"
                    elif dates[0]=="third ":
                        dates[0]="3" 
                    elif dates[0]=="third":
                        dates[0]="3"
                    elif dates[0]=="Third ":
                        dates[0]="3"
                    elif dates[0]=="Third":
                        dates[0]="3"
                    
                    elif dates[0]=="Tenth":
                        dates[0]="10" 
                    elif dates[0]=="Augusta":
                        dates[0]="9" 
                    elif (dates[0]=="this_sixth ")|(dates[0]=="sixth ")| (dates[0]=="Sixth") | (dates[0]=="Sixth ") |(dates[0]=="this sixth "):
                        dates[0]="6" 
                        
                    elif dates[0]=="Tenth ":
                        dates[0]="10"
                    elif (dates[0]=="twentieth")|(dates[0]=="This 20th "):
                        dates[0]="20"
                    elif (dates[0]=="twenty fourth ") |(dates[0]=="twenty fourth") |(dates[0]=="Twenty fourth ") |(dates[0]=="Twenty fourth"):
                        dates[0]="24"
                    elif (dates[0]=="fifteenth ") |(dates[0]=="Fifteenth ") |(dates[0]=="Fifteenth"):
                        dates[0]="15"
                    elif dates[0]=="twentieth ":
                        dates[0]="20"
                    elif dates[0]=="Eighteenth ":
                        dates[0]="18"
                    elif dates[0]=="Eighteenth":
                        dates[0]="18"
                    elif (dates[0]=="Nineteenth ") |(dates[0]=="Nineteenth"):
                        dates[0]="19"
                    elif dates[0]=="First ":
                        dates[0]="1"
                    elif dates[0]=="First":
                        dates[0]="1"
                    elif (dates[0]=="Sixteenth ")|(dates[0]=="this 16") |(dates[0]=="Sixteenth"):
                        dates[0]="16"
                   
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif dates[0]=="twenty third":
                        dates[0]="23"
                    elif dates[0]=="Twenty third ":
                        dates[0]="23"
                    elif dates[0]=="Twenty Second ":
                        dates[0]="22"
                    elif dates[0]=="Twenty Second":
                        dates[0]="22"
                    elif dates[0]=="Twenty Eighth ":
                        dates[0]="28"
                    elif dates[0]=="Twentysenventh ":
                        dates[0]="27"
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif (dates[0]=="twenty third") |(dates[0]=="Twentythird")|(dates[0]=="Twentythird "):
                        dates[0]="23"
                    elif (dates[0]=="fourteenth ") | (dates[0]=="fourteenth") | (dates[0]=="Fourteenth") | (dates[0]=="Fourteenth "):
                        dates[0]="14"
                    elif (dates[0]=="Eleventh ") | (dates[0]=="Eleventh") |(dates[0]=="eleventh ") :
                        dates[0]="11"
                    elif (dates[0]=="Thirteenth") | (dates[0]=="Thirteenth ") :
                        dates[0]="13"
                    elif (dates[0]=="seventeenth ")|(dates[0]=="Seventeenth") |(dates[0]=="seventeenth") |(dates[0]=="Seventeenth ") :
                        dates[0]="17"
                     
                     
                     
                    
                        
                    j = dates[0]+"-"+dates[1]+"-"+dates[2]
                    ng.append(j)
            
            
    NG.append(ng)   

In [9]:
Audf["Dates"]=NG

In [10]:
Audf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Dates
0,NaN,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]",[16-Sept-1865],[16-Septembre-1865]
1,NaN,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864],[],[]
2,NaN,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]",[3-October-1865],[3-October-1865]
3,NaN,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]",[],[]
4,NaN,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]",[31-May-1867],[31-May-1867]


In [11]:
Audf["text"]=range(2136)

In [12]:
len(NG)

2136

In [13]:
PL=[]
for i in NG:
    for j in i:
        PL.append(j)
len(PL)

3587

In [14]:
def uniqueName(iterable):
    seen = set()
    result = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result
NGG=[]
for i in NG:
    i=uniqueName(i)
    NGG.append(i)

In [15]:
PL=[]
for i in NGG:
    for j in i:
        PL.append(j)
len(PL)

2887

In [16]:

for i in NGG:
    for j in i:
        if j=="29-May-1865":
            i.remove(j)

In [17]:
for i in range(len(NGG)):
    if len(NGG[i])==2:
        NGG[i].remove(NGG[i][1])
    elif (len(NGG[i])>2):
        v=NGG[i][1]
        del(NGG[i][0:])
        NGG[i]=[v]
        

****

# dflarge

In [18]:
with open('../input/entities/entityL.pkl', 'rb') as f:
    entitiesL= pickle.load(f)

In [19]:
len(entitiesL)

2136

In [20]:
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dflarge=pd.DataFrame({"text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in entitiesL:
    dc={}
    P=[]
    L=[]
    G=[]
    O=[]
    for entity in i :

                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                       
        
      
    for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
    Per.append(P)
    ORG.append(O)
    LOC.append(L)
            

dflarge["PERSON-nerenglarge"]=Per
dflarge["LOC-nerenglarge"]=LOC
dflarge["ORG-nerenglarge"]=ORG   

In [21]:
dflarge.head()

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,NaN,"[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck County, State of North Carolina, Un...",[Constitution]
1,NaN,[],"[North Carolina, United States, States]","[Department of Virginia, OATH PAROLE]"
2,NaN,"[W. H. Pearce, Almighty, God, E. Ampetts, W . ...","[Craven County, State of NorthCarolina, United...",[Constitution]
3,NaN,[God],"[State of, North Carolina, United States]",[Constitution]
4,NaN,"[S.E, God, T.D, McAlpine]","[State of, North Carolina, Wake County, United...","[Constitution, Res . Corp]"


# final df

In [22]:
Audf["Person-NerEngLarge"]=dflarge["PERSON-nerenglarge"]
Audf["LOC-NerEngLarge"]=dflarge["LOC-nerenglarge"]
Audf["ORG-NerEngLarge"]=dflarge["ORG-nerenglarge"]

In [23]:
Audf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Dates,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge
0,0,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]",[16-Sept-1865],[16-Septembre-1865],"[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck County, State of North Carolina, Un...",[Constitution]
1,1,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864],[],[],[],"[North Carolina, United States, States]","[Department of Virginia, OATH PAROLE]"
2,2,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]",[3-October-1865],[3-October-1865],"[W. H. Pearce, Almighty, God, E. Ampetts, W . ...","[Craven County, State of NorthCarolina, United...",[Constitution]
3,3,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]",[],[],[God],"[State of, North Carolina, United States]",[Constitution]
4,4,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]",[31-May-1867],[31-May-1867],"[S.E, God, T.D, McAlpine]","[State of, North Carolina, Wake County, United...","[Constitution, Res . Corp]"


In [24]:
import os 
os.getcwd()

'/kaggle/working'

In [25]:
Audf.to_pickle("data.pkl")

In [26]:
with open('../input/audfff/data.pkl', 'rb') as f:
    data= pickle.load(f)

In [27]:
data

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Dates,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge
0,0,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]",[16-Sept-1865],[16-Septembre-1865],"[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck County, State of North Carolina, Un...",[Constitution]
1,1,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864],[],[],[],"[North Carolina, United States, States]","[Department of Virginia, OATH PAROLE]"
2,2,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]",[3-October-1865],[3-October-1865],"[W. H. Pearce, Almighty, God, E. Ampetts, W . ...","[Craven County, State of NorthCarolina, United...",[Constitution]
3,3,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]",[],[],[God],"[State of, North Carolina, United States]",[Constitution]
4,4,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]",[31-May-1867],[31-May-1867],"[S.E, God, T.D, McAlpine]","[State of, North Carolina, Wake County, United...","[Constitution, Res . Corp]"
...,...,...,...,...,...,...,...,...,...,...
2131,2131,"[Geo . Little, Geo . B. Ayer]","[Raleigh, N.C., Wake County, N.C, United State...",[],"[May 26, July 14]",[],[],"[Geo . Little, Geo, B. Ayer, Hugo Hillebrandt]","[Raleigh, N.C., Wake County, State of N.C, Uni...",[]
2132,2132,[Birnie],[],[],[July 30th 1867],[30-July-1867],[30-July-1867],[],[],[]
2133,2133,[William Birnie],"[North Carolina, Robeson County, United States]",[],[30 day of July],[],[],"[William Birnie, God, Wm Birnie, W.A]","[State of, North Carolina, Robeson County, Uni...",[Constitution]
2134,2134,"[A.L . Price, James Fulton, John D Conoley, Al...","[North Carolina, New Hanover County, New Hanov...",[Fulton Price],"[May 29 1865, 15th Sept 1865, 15th September A...","[29-May-1865, 15-Sept-1865, 15-September-1865,...","[29-May-1865, 15-Septembre-1865, 15-Septembre-...","[Price, James Fulton, Sgd W . N. Peden, Miles ...","[State of North Carolina, New Hanover County, ...","[Fulton Price, Confederate Government, Court o..."


# Separate Oath maker and witness name

In [28]:
newdata=pd.read_csv("../input/oathss/oaths.csv")

In [29]:
all_text=newdata["transcription_text"]
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punctuation
punct='!"#$%&\'()*+-/<=>?@[\\]^`{|}~'

In [30]:
print(all_text[433])

[[image - eagle]]
AMNESTY OATH. 

Bay St Louis Miss Sept 26th 1865.

I, Carl Ascher do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.

Carl Ascher

Sworn to and subscribed at Bay St Louis Miss }
this 26th day of September }
1865, before me, }
Rob W Norton J. P.

DESCRIPTION:
Age 53 years
Height 5 feet 5 inches
Color of Eyes Blue
Color of Hair Brown
Characteristics

Provost Marshal's Clerk

HEADQUARTERS Bay St Louis Mississippi
Sept 26th 1865.

I hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly administered to Carl Ascher of Bay St Louis by my order, this 26th day of September 1865.


In [31]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] 
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

for i in tqdm(range(newdata.shape[0])):
    all_text[i] = process(all_text[i])
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = " ".join(all_text[i])

100%|██████████| 2136/2136 [00:00<00:00, 3061.90it/s]


In [32]:
allPerson=[]
for i in Audf["PERSON-ontonotes"]:
            allPerson.append(i)
allDatess=[]
for i in Audf["Date-ontonotes"]:
            allDatess.append(i)
allpostdates=[]
for i in Audf["postDate-ontonotes"]:
            allpostdates.append(i)
allGPE=[]
for i in Audf["GPE-ontonotes"]:
            allGPE.append(i)
alORGl=[]
for i in Audf["ORG-ontonotes"]:
            alORGl.append(i)

In [33]:
Personlarge=[]
for i in dflarge["PERSON-nerenglarge"]:
            Personlarge.append(i)
ORGlarge=[]
for i in dflarge["ORG-nerenglarge"]:
            ORGlarge.append(i) 
LOClarge=[]
for i in dflarge["LOC-nerenglarge"]:
            LOClarge.append(i)

In [34]:
NewPerson=[]
for i in Personlarge:
    p=[]
    for j in i:
        j=j.replace('.','')
        j=j.replace('Oath','')
        j=j.replace('Sworn','')
        j=j.replace('Path of','')
        j=j.replace('Her X','')
        j=j.replace('  ',' ')
        p.append(j)
    NewPerson.append(p)
    
        

In [35]:
len(NewPerson)

2136

In [36]:
PL=[]
for i in NewPerson:
    for j in i:
        PL.append(j)
len(PL)

9512

# removing outliers

In [37]:
L=["God","GOD","god","P Mar","ANDREW JOHNSON","So Help Me God","FD","JC",'',"AB","DAS","CE","EC","andrew jonhson","Justice of Peace","Almighty God","Andrew Johnson","SE","TD","Jm","I","I J","MW","M","JH","JB","WD","JK","J P","C","G","HA","J","A","F A","DS","CT","RF",'JE',"Ashe","JL","H","JP","Mr","Sworn","sworn","JF","Bonnell","B","ER","WHé","JH","JW","F","Wm","W","WF"]

In [38]:
Per=[]
for i in NewPerson:
    p=[]
    for j in i:
        if j=="Almighty":
            j="Almighty God"
        if j not in L:
            p.append(j)
        
    Per.append(p)

In [39]:
PLl=[]
for i in Per:
    for j in i:
        PLl.append(j)
len(PLl)

5656

**from 9512 to 5663 great**

# removing redundant name

In [40]:
def uniqueName(iterable):
    seen = set()
    result = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result
Perr=[]
for i in Per:
    i=uniqueName(i)
    Perr.append(i)
    

In [41]:
PL=[]
for i in Perr:
    for j in i:
        PL.append(j)
len(PL)

5292

**from 5667 to 5299**

In [42]:
P=[]
for i in Perr:
    L=[]
    for j in range(len(i)):
        L.append(i[j].split())
    P.append(L)


In [43]:
Perr[0]

['I C Meekins', 'Isaac C Meekins', 'E H Walker']

In [44]:
P[0]

[['I', 'C', 'Meekins'], ['Isaac', 'C', 'Meekins'], ['E', 'H', 'Walker']]

In [45]:
P[20]

[['Nancy', 'Gaylord'],
 ['Nancy'],
 ['Gaylord'],
 ['B', 'Gilbert'],
 ['John', 'B'],
 ['J', 'P', 'Seal']]

In [46]:
# rp=[]
# d=[]
# f=[]
# for i in l:
#     r=[]
#     for j in range(len(i)-1):
#             if i[j][len(i[j])-1] == i[j+1][len(i[j+1])-1]:
#                 r.append(i[j])
#                 print(i[j][len(i[j])-1])
#             elif i[j][0]==i[j+1][0]:
#                 r.append(i[j+1])
#                 d.extend(i[j+1])
#             elif i[j][len(i[j])-1]==i[j+1][0]:
#                 r.append(i[j+1])
#                 f.extend([i[j],i[j+1]])
#                 print(i[j][len(i[j])-1])
                
#     rp.append(r)

In [47]:
RP=[]
d=[]
f=[]
for i in P:
    r=[]
    for j in range(len(i)-1):
            if i[j][len(i[j])-1] == i[j+1][len(i[j+1])-1]:
                r.append(i[j])
            elif i[j][0]==i[j+1][0]:
                r.append(i[j+1])
                d.extend([i[j],i[j+1]])
            elif i[j][len(i[j])-1]==i[j+1][0]:
                r.append(i[j+1])
                f.extend([i[j],i[j+1]])
    RP.append(r)

In [48]:
RP[9]

[['T', 'A', 'Dough']]

In [49]:
R=[]
for i in RP:
    pr=[]
    for j in tqdm(i):
       
        j=" ".join(j)
        pr.append(j)
    R.append(pr)

100%|██████████| 1/1 [00:00<00:00, 2866.92it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 13662.23it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 17189.77it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 17403.75it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 14413.42it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 16710.37it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 16644.06it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, 

In [50]:
P1=[]
for i in P:
    pr=[]
    for j in tqdm(i):
       
        j=" ".join(j)
        pr.append(j)
    P1.append(pr)

100%|██████████| 3/3 [00:00<00:00, 37673.39it/s]
0it [00:00, ?it/s]
100%|██████████| 3/3 [00:00<00:00, 26490.34it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 15420.24it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 29026.33it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 31536.12it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 31655.12it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 22550.02it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 26800.66it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 21788.59it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 14074.85it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 10810.06it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 15196.75it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 15420.24it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 32388.45it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 7928.74it/s]


In [51]:
all_text[20]

'I Nancy Gaylord of Currituck County State of North Carolina solemnly swear affirm presence of Almighty God , I henceforth faithfully support protect defend Constitution of United States union of States thereunder ; I like manner abide faithfully support law proclamation made existing rebellion reference emancipation of slave help God , Nancy Her X mark Gaylord Witness : Mr B. Gilbert . Sworn subscribed 20th day of October , A.D. 1866 . John B. J. P. Seal'

In [52]:
P1[20]

['Nancy Gaylord', 'Nancy', 'Gaylord', 'B Gilbert', 'John B', 'J P Seal']

In [53]:
FinalPersonlst=[]
for i in P1:
    p=[]
    for j in range(len(i)):
        if i[j] not in R[P1.index(i)]:
            p.append(i[j])
    FinalPersonlst.append(p)
            

In [54]:
FinalPersonlst[20]

['Nancy Gaylord', 'Gaylord', 'B Gilbert', 'John B', 'J P Seal']

In [55]:
FinalPersonlst[20:40]

[['Nancy Gaylord', 'Gaylord', 'B Gilbert', 'John B', 'J P Seal'],
 ['Martin Hubbard', 'C Merkins'],
 ['Rudolph Ebecke', 'G Badger Harris'],
 ['Thomas C McIlhenny', 'James F'],
 ['J M Nilsons'],
 ['Matilda Miller', 'Miller', 'John B'],
 ['Hugo Hillebrandt'],
 ['George M Adams'],
 [],
 ['John H'],
 ['John H Archdeacon', 'Jacob F Chur'],
 ['W Arnold'],
 ['GW Arnold', 'John J Conoly'],
 ['John D Black'],
 ['Jno D Black', 'Jacob F Chur'],
 [],
 ['William Birnie', 'W A Dick JP'],
 ['William Birnie'],
 ['Wm Birnie', 'Jacob F Chur'],
 []]

In [56]:
total=[]
for i in FinalPersonlst:
    for j in i:
        total.append(j)
len(total)

4622

**4767 names !** <br>
**4630 names !**

In [57]:

for i in FinalPersonlst:
    for j in range(len(i)):
        if i[j] =="McAlpine":
            i[j]="McAlpine Lt VRC"
        elif i[j]=="W P Scott":
            i[j]="W P Scott Capt"

In [58]:
all_text[1875]

'Copy Washington Oct. 2nd 1865 Hon . William H. Seward Secretary of State : Sir : I honor acknowledge receipt of Presidents Warrant of Pardon bearing date 21st of July , 1865 , hereby signify acceptance of condition therein specified . Very Respectfully Yr. Obdt . Servt . John Screven State of Georgia County of Chatham I , Dominick A. OByrne , Ordinary of Chatham County ex Officio Clerk of court , Do Hereby Certify annexed foregoing instrument of writing true faithful copy of original letter of John Screven acknowledging received Presidents Warrant of Pardon signifying acceptance of . In testimony whereof I hereunto set hand affixed seal Court of Ordinary 19th day of October 1865 . Dominick A OByrne Ordinary Ex officio Clerk of Court of Ordinary of Chatham County'

In [59]:
FinalPersonlst[1875]

['William H Seward', 'John Screven', 'Dominick A']

In [60]:
nb0=0
nb1=0
nb2=0
nb3=0
nb4=0
nb7=0
for i in FinalPersonlst:
    if i==[]:
        nb0=nb0+1
    if len(i)==1:
        nb1=nb1+1
    if len(i)==2:
        nb2=nb2+1
    if len(i)==3:
        nb3=nb3+1
    if len(i)==4:
        nb4=nb4+1
    elif len(i)>4:
        nb7=nb7+1
        

In [61]:
nb0,nb1,nb2,nb3,nb4,nb7

(141, 383, 949, 446, 141, 76)

In [62]:
# nb+nb1+nb2+nb3+nb4+nb7

In [63]:
FinalPersonlst[433]

['Carl Ascher', 'Rob W Norton', 'Frank E Miller']

In [64]:
FinalPersonlst[500:510]

[['J H Mathews', 'Frank E Miller'],
 ['William Collins', 'JH Mathew'],
 ['Wm X mark Collins', 'Chas W Kirkland', 'Wm Collins'],
 ['Henry Colligan', 'J H Mathews', 'Marshall'],
 ['Henry Coligan', 'J H Mathews', 'Chas W Kirkland'],
 ['Peter Semore', 'J H Mathews'],
 ['Peter X mark Semore', 'J H Mathews', 'Chas W Kirkland'],
 ['Wm Getchell'],
 ['Hubbard Crawfoot', 'William Getchell', 'Frank E Miller'],
 ['Eliha Carver', 'Robt W Norton']]

# extract Oath_maker names and witness_names from FinalPersonlst 

* > 1: oath maker 
* > 2: first : Oath maker , second : wwitness name
* > 3: first : IOath maker , third : witness name
* > 4: first : Oath maker , fourth : whitness name  

In [65]:
Oath_maker=[]
wit_names=[]
for i in FinalPersonlst:
    if len(i)==1:
        Oath_maker.append(i[0])
        wit_names.append('')
    elif len(i)==2:
        Oath_maker.append(i[0])
        wit_names.append(i[1])
    elif len(i)==0:
        Oath_maker.append('')
        wit_names.append('')
        
    else:
        Oath_maker.append(i[0])
        wit_names.append(i[len(i)-1])
    

In [66]:
len(Oath_maker), len(wit_names)

(2136, 2136)

In [67]:
nb=0
for i in wit_names:
    if i !="":
        nb=nb+1
nb

1612

**1613 witness name**

In [68]:
wit_names[1500:1520]

['Sanders',
 'Fredk A Ford',
 'Jos O Hear Sanders',
 'Miss E Sanders',
 'J Milton Thompson',
 'J Chase Jr',
 'Milton Thompson',
 'I B W Seabrooks',
 'John S Dudley',
 '',
 'Levi Stuber',
 'James Entwisle',
 'Levi Stuber',
 'Levi Stuber',
 'Levi Stuber',
 'Levi Stuber',
 'Ed N Ketchum',
 'Ed N Ketchum',
 'Levi Stuber',
 'N R Reed']

In [69]:
nb=0
for i in Oath_maker:
    if i !="":
        nb=nb+1
nb

1995

**1995 oath maker !**

In [70]:
Oath_maker[150:170]

['Sydney',
 'John Lawson',
 'John X Mark Lawson',
 'Wm J Dawes',
 'E B Lumberton',
 'Jno Gedes',
 'Jany',
 '',
 'T D McAlpine',
 'W Harris',
 'Mary A',
 'Mary A',
 '',
 'Gardner Merriam',
 'Jacob F Chur',
 'T A Mitchell',
 'I T',
 'Jacob F Chur',
 'McFarland',
 'John W Leak']

In [71]:
wit_names

['E H Walker',
 '',
 'JP A B Watson',
 '',
 '',
 'Isaac C Merkins',
 'JM Wilson',
 'J P John B Etheridge',
 'A D Etheridge',
 'J P E H Walker',
 'John N Niles',
 'John N Niles',
 'Chadwick',
 'J P H Rountree',
 'M F Arendell',
 'J Cochran',
 '',
 '',
 '',
 'J M Wilson',
 'J P Seal',
 'C Merkins',
 'G Badger Harris',
 'James F',
 '',
 'John B',
 '',
 '',
 '',
 '',
 'Jacob F Chur',
 '',
 'John J Conoly',
 '',
 'Jacob F Chur',
 '',
 'W A Dick JP',
 '',
 'Jacob F Chur',
 '',
 'John R Estes',
 '',
 'M Cogswell',
 '',
 'Calvin C Jones',
 '',
 'T D McAlpine',
 '',
 'AW McKillip',
 '',
 'Jas C Davis',
 '',
 'Wm G Forbes',
 '',
 'F S Wiatt JP',
 '',
 'Wm Maxwell',
 '',
 '',
 '',
 'Jacob F Chur',
 '',
 '',
 '',
 'BuRF AL',
 '',
 '',
 '',
 'Chas Hibbard',
 '',
 '',
 '',
 'A Coats',
 '',
 'B R F A L AP White',
 '',
 'C W Dodge',
 '',
 '',
 '',
 'Jacob F Chur',
 '',
 'Jacob F Chur',
 '',
 '',
 '',
 'Stephen Moore',
 '',
 '',
 '',
 '186 W R Young',
 '',
 'Clinton A Cilley',
 '',
 '',
 '',
 '',
 '',


##### 